# **Sarvam Doctranslate API Tutorial**

This notebook demonstrates how to use the **Sarvam Doctranslate API** to translate PDF files from one language to another. The API supports custom translation dictionaries and allows you to translate specific pages of a PDF.



# **1. Installation**

Before you begin, ensure you have the necessary Python libraries installed. Run the following commands to install the required packages:


In [6]:
!pip install requests ipywidgets

# **2. Setting Up the API Key**

To use the Sarvam Doctranslate API, you need an API subscription key. Follow these steps to set up your API key:

1. **Obtain your API key**: If you don’t have an API key, sign up on the [Sarvam AI platform](https://dashboard.sarvam.ai) to get one.
2. **Enter your API key**: In the UI widget below, enter your API key in the "API Key" field.

---

# **3. Uploading PDF Files**

To translate a PDF file, you need to upload it to the notebook. Follow these steps:

1. **Prepare your PDF file**: Ensure your file is in `.pdf` format.
2. **Upload the file**: Use the file upload widget below to upload your PDF file.


# **4. Using the Sarvam Doctranslate API**

This section demonstrates how to use the Sarvam Doctranslate API to translate a PDF file.

## **4.1. Defining the `translate_pdf` Function**

The `translate_pdf` function sends the PDF file to the Sarvam API and retrieves the translated PDF.

In [7]:
import requests
import base64
import json
import tempfile
import os

def translate_pdf(pdf_file_path, page_number, input_lang, output_lang, hard_translate_dict, api_key):
    """
    Translates a PDF file using the Sarvam Doctranslate API.

    Args:
        pdf_file_path (str): Path to the PDF file.
        page_number (int): Page number to translate (1-based index).
        input_lang (str): Input language code (e.g., 'en-IN' for English).
        output_lang (str): Output language code (e.g., 'hi-IN' for Hindi).
        hard_translate_dict (dict): Custom translation dictionary (optional).
        api_key (str): Your Sarvam API subscription key.

    Returns:
        str: Path to the translated PDF file or an error message.
    """
    url = "https://api.sarvam.ai/parse/translatepdf"  # Doctranslate API URL
    headers = {'api-subscription-key': api_key}

    try:
        with open(pdf_file_path, 'rb') as pdf_file:
            files = {
                'pdf': pdf_file,  # Upload the PDF file for translation
                'page_number': (None, str(page_number)),
                'input_lang': (None, input_lang),
                'output_lang': (None, output_lang),
            }

            # If a custom translation dictionary is provided
            if hard_translate_dict:
                files['hard_translate_dict'] = (None, json.dumps(hard_translate_dict))

            # Send POST request to the API
            response = requests.post(url, headers=headers, files=files)

            # Check if the request was successful
            if response.status_code == 200:
                translated_pdf_base64 = response.json().get('translated_pdf', '')
                if translated_pdf_base64:
                    # Decode base64 to raw PDF
                    decoded_pdf = base64.b64decode(translated_pdf_base64)
                    output_pdf_path = "translated_output.pdf"

                    # Save the translated PDF locally
                    with open(output_pdf_path, "wb") as file:
                        file.write(decoded_pdf)

                    return output_pdf_path
                else:
                    return "Translation failed. No data returned."
            else:
                return f"❌ API Error ({response.status_code}): {response.text}"

    except Exception as e:
        return f"⚠️ An error occurred: {str(e)}"


## **4.2. Creating the UI Widgets**

The following code creates interactive UI widgets for uploading the PDF file, selecting the page number, specifying input and output languages, providing a custom translation dictionary, and entering the API key.

In [8]:
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import files

# UI Widgets
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
page_number_widget = widgets.IntText(value=1, description="Page No:")
input_lang_widget = widgets.Text(value='en-IN', description="Input Lang:")
output_lang_widget = widgets.Text(value='hi-IN', description="Output Lang:")
hard_translate_dict_widget = widgets.Text(description="Custom Dict (JSON format):")
api_key_widget = widgets.Password(description="API Key:")
parse_button = widgets.Button(description="Translate PDF", button_style="primary")


## **4.3. Handling PDF Translation**

The `handle_translate` function processes the uploaded PDF file, calls the Sarvam API, and saves the translated PDF.

In [11]:
def handle_translate(_):
    """
    Handles the PDF translation process when the "Translate PDF" button is clicked.
    """
    if upload_widget.value:
        uploaded_filename = list(upload_widget.value.keys())[0]
        file_content = upload_widget.value[uploaded_filename]['content']

        # Save the uploaded file temporarily
        temp_pdf_path = os.path.join(tempfile.gettempdir(), uploaded_filename)
        with open(temp_pdf_path, "wb") as temp_file:
            temp_file.write(file_content)

        # Get user inputs
        page_number = page_number_widget.value
        input_lang = input_lang_widget.value
        output_lang = output_lang_widget.value
        hard_translate_dict_input = hard_translate_dict_widget.value
        hard_translate_dict = {}

        # Optionally parse the custom dictionary
        if hard_translate_dict_input:
            try:
                hard_translate_dict = json.loads(hard_translate_dict_input)
            except json.JSONDecodeError:
                display(HTML("<p style='color:red;'>❌ Invalid dictionary format. Proceeding without custom dictionary.</p>"))

        api_key = api_key_widget.value

        # Display processing message
        display(HTML(f"<p>📄 Translating <b>{uploaded_filename}</b>... Please wait.</p>"))

        # Call the API to translate the PDF
        translated_pdf_path = translate_pdf(temp_pdf_path, page_number, input_lang, output_lang, hard_translate_dict, api_key)

        if translated_pdf_path.startswith("translated_output.pdf"):
            # Provide a download link
            display(HTML(f"""
                <p>✅ <b>Translation complete!</b> Your translated PDF has been downloaded.</p>
            """))
            files.download(translated_pdf_path)
        else:
            # Display the error message
            display(HTML(f"<p style='color:red;'>❌ {translated_pdf_path}</p>"))
# Bind button click event
parse_button.on_click(handle_translate)

### **4.4. Displaying the UI**

This section displays the UI widgets for uploading the PDF file, entering the page number, specifying input and output languages, providing a custom translation dictionary, and entering the API key.

In [12]:
# Display UI
display(HTML("<h3>📄 Sarvam Doctranslate PDF Translator</h3><p>Upload a PDF file, enter translation details, and click 'Translate PDF' to start the translation.</p>"))
display(upload_widget, page_number_widget, input_lang_widget, output_lang_widget, hard_translate_dict_widget, api_key_widget, parse_button)

FileUpload(value={'who_moved_my_cheese.pdf': {'metadata': {'name': 'who_moved_my_cheese.pdf', 'type': 'applica…

IntText(value=3, description='Page No:')

Text(value='en-IN', description='Input Lang:')

Text(value='hi-IN', description='Output Lang:')

Text(value='{"Who moved my cheese?" : "नमस्कार"}', description='Custom Dict (JSON format):')

...

Button(button_style='primary', description='Translate PDF', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **5. Conclusion**

This notebook demonstrated how to use the **Sarvam Doctranslate API** to translate PDF files from one language to another. By following the steps, you can:

1. Upload a PDF file.
2. Translate the file using the Sarvam API.
3. Download the translated PDF for further use.

---

# **6. Additional Resources**

For more details, refer to the official **Sarvam API documentation** and join the community for support:

- **Documentation**: [docs.sarvam.ai](https://docs.sarvam.ai)  
- **Community**: [Join the Discord Community](https://discord.gg/hTuVuPNF)

---

# **7. Final Notes**

- Keep your API key secure.
- Use the appropriate input and output language codes (e.g., `en-IN` for English, `hi-IN` for Hindi).
- Explore advanced features like custom translation dictionaries for specialized terminology.

Happy translating! 🚀